In [101]:
import pandas as pd
import uuid
import numpy as np

caminho_arquivo = r'C:\Users\User\Desktop\Ada Hackaton\adahack-2024-dados\base_dados\base_de_dados.csv'
df = pd.read_csv(caminho_arquivo)


In [102]:
# Lista de substituições
substituicoes = {
    'Ã©': 'é',
    'Ã¢': 'â',
    'Ãº': 'ú',
    'Ã­': 'í',
    'Ã§': 'ç',
    'Ã¡': 'á',
    'Ã³': 'ó',
    'Ãª': 'ê',
    'Ã®': 'î',
    'Ã´': 'ô',
    'Ã»': 'û',
    'Ã£': 'ã',
    'Ãµ': 'õ',
    'Ã': 'à',
    'Ã¨': 'è',
    'Ã¬': 'ì',
    'Ã¹': 'ù',
    'Ã¤': 'ä',
    'Ã«': 'ë',
    'Ã¯': 'ï',
    'Ã¶': 'ö',
    'Ã¼': 'ü'
}

# Converte todas as colunas para o tipo de dado string
df = df.astype(str)

# Aplica as substituições em todas as colunas do DataFrame
for coluna in df.columns:
    for antigo, novo in substituicoes.items():
        df[coluna] = df[coluna].str.replace(antigo, novo)

# Exibe o DataFrame após a limpeza
display(df)

,id,id.1,Nome,Genero,Idade,RaÃ§a,EndereÃ§o,FormaÃ§Ã£o,Estado,Tempo de casa,Departamento,Senioridade
0,1,1,Eleonora Arilda Penedo Gomes de Padilha,Fem,34.0,pardo,"9155 Harold Oval\r\nSellersside, FL 21337",Ensino Médio,Santa Catarina,12.0,Compras,Analista Pleno
1,2,2,Elisângela Gabrielle de Osório,Fem,26.0,pardo,"941 Martin Manor\r\nLake Isaiahtown, FM 43797",Ensino Médio,Pará,6.0,Contabilidade,Analista Júnior
2,3,3,José Túlio de Cabral,Masc,35.0,pardo,"110 Davis Ridges\r\nMejiaville, LA 17095",Ensino Médio,Santa Catarina,5.0,Vendas,Analista Pleno
3,4,4,Ezequiel Edivaldo de Medeiros Sonao,Masc,24.0,pardo,"48010 Wilson Glen Apt. 749\r\nSmithborough, NV...",Ensino Superior,Tocantins,4.0,Administrativo,Gerente
4,5,5,Fagner Josiel dos Santos,Masc,21.0,pardo,"8666 Ramos Ports Apt. 070\r\nSandraport, MN 33570",Ensino Superior,Ceará,5.0,Recursos Humanos,Analista Júnior
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,9996,Omar Camilo da Paz,Masc,21.0,pardo,"47323 Casey Junction\r\nJaimefort, UT 49158",Ensino Superior,Rondônia,9.0,Contabilidade,Analista Júnior
9996,9997,9997,Edu de Muniz Jinuyul Neto,Masc,44.0,pardo,"1431 Gina Gardens Apt. 534\r\nJohnsonshire, GU...",Ensino Médio,Paraíba,10.0,Contabilidade,Analista Júnior
9997,9998,9998,Benedito Robert Dlievic,Masc,21.0,branco,"41179 Tracie Parkways\r\nLangfort, IA 88137",Ensino Superior,Distrito Federal,6.0,Desenvolvimento de Produtos,Gerente
9998,9999,9999,Bento Osvaldo do Piauí,Masc,39.0,pardo,"902 Luis Island\r\nBrownton, NJ 61984",Ensino Superior,Paraná,5.0,Vendas,Analista Júnior


In [103]:
# Mostra os nomes das colunas antes da correção
print("Nomes das Colunas antes da Correção:")
print(df.columns)

# Renomeando as colunas corrompidas
df = df.rename(columns={
    'RaÃ§a': 'Raça',
    'EndereÃ§o': 'Endereço',
    'FormaÃ§Ã£o': 'Formação'
})

# Mostra os nomes das colunas após a correção
print("\nNomes das Colunas após a Correção:")
print(df.columns)

Nomes das Colunas antes da Correção:
Index(['id', 'id.1', 'Nome', 'Genero', 'Idade', 'RaÃ§a', 'EndereÃ§o',
       'FormaÃ§Ã£o', 'Estado', 'Tempo de casa', 'Departamento', 'Senioridade'],
      dtype='object')

Nomes das Colunas após a Correção:
Index(['id', 'id.1', 'Nome', 'Genero', 'Idade', 'Raça', 'Endereço', 'Formação',
       'Estado', 'Tempo de casa', 'Departamento', 'Senioridade'],
      dtype='object')


In [104]:
# Removendo a coluna "id"
df.drop(columns=['id', 'id.1'], inplace=True)  # Remove as colunas "id" e "id.1"

# Função para gerar IDs únicos de 4 dígitos (letras e números)
def gerar_id():
    return str(uuid.uuid4().hex[:4]).upper()

# Criando uma nova coluna "ID" com IDs únicos de 4 dígitos
df['ID'] = [gerar_id() for _ in range(len(df))]

# Reorganizando as colunas para ter a nova coluna "ID" como a primeira
colunas = df.columns.tolist()
colunas = ['ID'] + [coluna for coluna in colunas if coluna != 'ID']
df = df[colunas]

# Mostra o DataFrame com a nova coluna "ID"
print(df.head())

     ID                                     Nome Genero Idade   Raça  \
0  6A4F  Eleonora Arilda Penedo Gomes de Padilha    Fem  34.0  pardo   
1  B06F           Elisângela Gabrielle de Osório    Fem  26.0  pardo   
2  DA20                     José Túlio de Cabral   Masc  35.0  pardo   
3  DF2D      Ezequiel Edivaldo de Medeiros Sonao   Masc  24.0  pardo   
4  9413                 Fagner Josiel dos Santos   Masc  21.0  pardo   

                                            Endereço         Formação  \
0          9155 Harold Oval\r\nSellersside, FL 21337     Ensino Médio   
1      941 Martin Manor\r\nLake Isaiahtown, FM 43797     Ensino Médio   
2           110 Davis Ridges\r\nMejiaville, LA 17095     Ensino Médio   
3  48010 Wilson Glen Apt. 749\r\nSmithborough, NV...  Ensino Superior   
4  8666 Ramos Ports Apt. 070\r\nSandraport, MN 33570  Ensino Superior   

           Estado Tempo de casa      Departamento      Senioridade  
0  Santa Catarina          12.0           Compras   Analist

In [105]:
# Convertendo a coluna 'Idade' para números numéricos
df['Idade'] = pd.to_numeric(df['Idade'], errors='coerce')

# Substituindo idades menores que 14 anos por NaN
df.loc[df['Idade'] < 14, 'Idade'] = np.nan

# Convertendo as idades para números inteiros
df['Idade'] = df['Idade'].astype('Int64')

# Mostra o DataFrame com as idades corrigidas
print(df.head())

     ID                                     Nome Genero  Idade   Raça  \
0  6A4F  Eleonora Arilda Penedo Gomes de Padilha    Fem     34  pardo   
1  B06F           Elisângela Gabrielle de Osório    Fem     26  pardo   
2  DA20                     José Túlio de Cabral   Masc     35  pardo   
3  DF2D      Ezequiel Edivaldo de Medeiros Sonao   Masc     24  pardo   
4  9413                 Fagner Josiel dos Santos   Masc     21  pardo   

                                            Endereço         Formação  \
0          9155 Harold Oval\r\nSellersside, FL 21337     Ensino Médio   
1      941 Martin Manor\r\nLake Isaiahtown, FM 43797     Ensino Médio   
2           110 Davis Ridges\r\nMejiaville, LA 17095     Ensino Médio   
3  48010 Wilson Glen Apt. 749\r\nSmithborough, NV...  Ensino Superior   
4  8666 Ramos Ports Apt. 070\r\nSandraport, MN 33570  Ensino Superior   

           Estado Tempo de casa      Departamento      Senioridade  
0  Santa Catarina          12.0           Compras   A

In [106]:
# Convertendo a coluna 'Tempo de casa' para numérico
df['Tempo de casa'] = pd.to_numeric(df['Tempo de casa'], errors='coerce')

# Corrigindo o tempo de casa para nulo se for maior que a idade
df.loc[df['Tempo de casa'] > df['Idade'], 'Tempo de casa'] = np.nan

# Convertendo as idades e tempos de casa para números inteiros
df['Idade'] = df['Idade'].astype('Int64')
df['Tempo de casa'] = df['Tempo de casa'].astype('Int64')

# Mostra o DataFrame com as correções
print(df.head())

     ID                                     Nome Genero  Idade   Raça  \
0  6A4F  Eleonora Arilda Penedo Gomes de Padilha    Fem     34  pardo   
1  B06F           Elisângela Gabrielle de Osório    Fem     26  pardo   
2  DA20                     José Túlio de Cabral   Masc     35  pardo   
3  DF2D      Ezequiel Edivaldo de Medeiros Sonao   Masc     24  pardo   
4  9413                 Fagner Josiel dos Santos   Masc     21  pardo   

                                            Endereço         Formação  \
0          9155 Harold Oval\r\nSellersside, FL 21337     Ensino Médio   
1      941 Martin Manor\r\nLake Isaiahtown, FM 43797     Ensino Médio   
2           110 Davis Ridges\r\nMejiaville, LA 17095     Ensino Médio   
3  48010 Wilson Glen Apt. 749\r\nSmithborough, NV...  Ensino Superior   
4  8666 Ramos Ports Apt. 070\r\nSandraport, MN 33570  Ensino Superior   

           Estado  Tempo de casa      Departamento      Senioridade  
0  Santa Catarina             12           Compras  

In [107]:
# Dicionário de mapeamento de estados
mapeamento_estados = {
    'Acre': 'AC', 'Alagoas': 'AL', 'Amapá': 'AP', 'Amazonas': 'AM', 'Bahia': 'BA',
    'Ceará': 'CE', 'Distrito Federal': 'DF', 'Espírito Santo': 'ES', 'Goiás': 'GO',
    'Maranhão': 'MA', 'Mato Grosso': 'MT', 'Mato Grosso do Sul': 'MS', 'Minas Gerais': 'MG',
    'Pará': 'PA', 'Paraíba': 'PB', 'Paraná': 'PR', 'Pernambuco': 'PE', 'Piauí': 'PI',
    'Rio de Janeiro': 'RJ', 'Rio Grande do Norte': 'RN', 'Rio Grande do Sul': 'RS',
    'Rondônia': 'RO', 'Roraima': 'RR', 'Santa Catarina': 'SC', 'São Paulo': 'SP',
    'Sergipe': 'SE', 'Tocantins': 'TO'
}

# Aplicando o mapeamento de estados
df['Estado'] = df['Estado'].map(mapeamento_estados)

# Mostra o DataFrame com as siglas dos estados
print(df.head())

     ID                                     Nome Genero  Idade   Raça  \
0  6A4F  Eleonora Arilda Penedo Gomes de Padilha    Fem     34  pardo   
1  B06F           Elisângela Gabrielle de Osório    Fem     26  pardo   
2  DA20                     José Túlio de Cabral   Masc     35  pardo   
3  DF2D      Ezequiel Edivaldo de Medeiros Sonao   Masc     24  pardo   
4  9413                 Fagner Josiel dos Santos   Masc     21  pardo   

                                            Endereço         Formação Estado  \
0          9155 Harold Oval\r\nSellersside, FL 21337     Ensino Médio     SC   
1      941 Martin Manor\r\nLake Isaiahtown, FM 43797     Ensino Médio     PA   
2           110 Davis Ridges\r\nMejiaville, LA 17095     Ensino Médio     SC   
3  48010 Wilson Glen Apt. 749\r\nSmithborough, NV...  Ensino Superior     TO   
4  8666 Ramos Ports Apt. 070\r\nSandraport, MN 33570  Ensino Superior     CE   

   Tempo de casa      Departamento      Senioridade  
0             12          

In [108]:
# Mapeamento de gêneros
mapeamento_genero = {
    'Masc': 'Masculino',
    'Fem': 'Feminino',
    'Neutro': 'Neutro',
    'Outros': 'Outros',
    np.nan: 'Não informado'
}

# Aplicando o mapeamento na coluna 'Genero' e substituindo NaN por 'Não informado'
df['Genero'] = df['Genero'].map(mapeamento_genero)

# Mostra o DataFrame com a coluna 'Genero' ajustada
display(df.head())

,ID,Nome,Genero,Idade,Raça,Endereço,Formação,Estado,Tempo de casa,Departamento,Senioridade
0,6A4F,Eleonora Arilda Penedo Gomes de Padilha,Feminino,34,pardo,"9155 Harold Oval\r\nSellersside, FL 21337",Ensino Médio,SC,12,Compras,Analista Pleno
1,B06F,Elisângela Gabrielle de Osório,Feminino,26,pardo,"941 Martin Manor\r\nLake Isaiahtown, FM 43797",Ensino Médio,PA,6,Contabilidade,Analista Júnior
2,DA20,José Túlio de Cabral,Masculino,35,pardo,"110 Davis Ridges\r\nMejiaville, LA 17095",Ensino Médio,SC,5,Vendas,Analista Pleno
3,DF2D,Ezequiel Edivaldo de Medeiros Sonao,Masculino,24,pardo,"48010 Wilson Glen Apt. 749\r\nSmithborough, NV...",Ensino Superior,TO,4,Administrativo,Gerente
4,9413,Fagner Josiel dos Santos,Masculino,21,pardo,"8666 Ramos Ports Apt. 070\r\nSandraport, MN 33570",Ensino Superior,CE,5,Recursos Humanos,Analista Júnior


In [109]:
# Corrigindo a coluna 'Raça' para ter a primeira letra em maiúscula e valores nulos substituídos
df['Raça'] = df['Raça'].fillna('Não informado').apply(lambda x: x.capitalize() if not pd.isnull(x) else x)

# Mostra o DataFrame com a coluna 'Raça' corrigida
display(df.head())

,ID,Nome,Genero,Idade,Raça,Endereço,Formação,Estado,Tempo de casa,Departamento,Senioridade
0,6A4F,Eleonora Arilda Penedo Gomes de Padilha,Feminino,34,Pardo,"9155 Harold Oval\r\nSellersside, FL 21337",Ensino Médio,SC,12,Compras,Analista Pleno
1,B06F,Elisângela Gabrielle de Osório,Feminino,26,Pardo,"941 Martin Manor\r\nLake Isaiahtown, FM 43797",Ensino Médio,PA,6,Contabilidade,Analista Júnior
2,DA20,José Túlio de Cabral,Masculino,35,Pardo,"110 Davis Ridges\r\nMejiaville, LA 17095",Ensino Médio,SC,5,Vendas,Analista Pleno
3,DF2D,Ezequiel Edivaldo de Medeiros Sonao,Masculino,24,Pardo,"48010 Wilson Glen Apt. 749\r\nSmithborough, NV...",Ensino Superior,TO,4,Administrativo,Gerente
4,9413,Fagner Josiel dos Santos,Masculino,21,Pardo,"8666 Ramos Ports Apt. 070\r\nSandraport, MN 33570",Ensino Superior,CE,5,Recursos Humanos,Analista Júnior


In [110]:
linhas_duplicadas = df.duplicated()

total_linhas_duplicadas = linhas_duplicadas.sum()

if total_linhas_duplicadas > 0:
    print("Existem", total_linhas_duplicadas, "linhas duplicadas no Dataset.")
else:
    print("Não existem linhas duplicadas no Dataset.")

Não existem linhas duplicadas no Dataset.


In [111]:
valores_distintos = df['Genero'].unique()
print(valores_distintos)

['Feminino' 'Masculino']


In [112]:
df.drop(columns=['Endereço'], inplace=True)

In [113]:
df.to_csv("base_de_dados_limpa.csv")